# Coropleth map

In [2]:
import pandas as pd
import duckdb

df = pd.read_excel(
    "data/resultat-ansokningsomgang-2024.xlsx", sheet_name="Tabell 3", skiprows=5
)
df.head()

,Utbildningsområde,SUN5 inriktning,SUN5 inriktning namn,Utbildningsnamn,Beslut,Diarienummer,Flera kommuner,Antal kommuner,Län,Kommun,...,Sökta utbildningsomgångar,Beviljade utbildningsomgångar,Sökta platser per utbildningsomgång,Sökta platser totalt,Beviljade platser utbildningsomgång 1,Beviljade platser utbildningsomgång 2,Beviljade platser utbildningsomgång 3,Beviljade platser utbildningsomgång 4,Beviljade platser utbildningsomgång 5,Beviljade platser totalt
0,Data/IT,481ab,"Utbildningar till programmerare, spel",AI Programmer,Avslag,MYH 2024/3742,Nej,1,Västerbotten,Umeå,...,3,0,30,90,0,0,0,0,0,0
1,Data/IT,481ab,"Utbildningar till programmerare, spel","Forsbergs, Spelutvecklare – Game Programming",Avslag,MYH 2024/4174,Nej,1,Stockholm,Stockholm,...,3,0,20,60,0,0,0,0,0,0
2,Data/IT,481ab,"Utbildningar till programmerare, spel",Futuregames Game Programmer Specialization,Avslag,MYH 2024/3029,Ja,2,Flera kommuner,Flera kommuner,...,3,0,30,90,0,0,0,0,0,0
3,Data/IT,481ab,"Utbildningar till programmerare, spel",Game Programmer,Avslag,MYH 2024/3425,Nej,1,Blekinge,Karlshamn,...,3,0,25,75,0,0,0,0,0,0
4,Data/IT,481ab,"Utbildningar till programmerare, spel",Spelprogrammerare,Beviljad,MYH 2024/3852,Nej,1,Skåne,Malmö,...,4,3,35,140,35,35,35,0,0,105


In [3]:
decisions = df["Beslut"].value_counts()
approved, total = decisions["Beviljad"], decisions.sum()
approved, total
                    

(np.int64(344), np.int64(1272))

In [4]:
df_regions = duckdb.query(
    """
    SELECT 
        län,
        CAST(COUNT_IF(beslut = 'Beviljad') AS INTEGER) AS Beviljade
    FROM df 
    WHERE län != 'Flera kommuner'
    GROUP BY län
    ORDER BY 
        beviljade
    DESC
"""
).df()

df_regions


,Län,Beviljade
0,Stockholm,82
1,Västra Götaland,63
2,Skåne,37
3,Östergötland,16
4,Jönköping,13
5,Örebro,10
6,Västernorrland,6
7,Kalmar,5
8,Kronoberg,5
9,Västmanland,5


In [5]:
df_regions["Län"].values

array(['Stockholm', 'Västra Götaland', 'Skåne', 'Östergötland',
       'Jönköping', 'Örebro', 'Västernorrland', 'Kalmar', 'Kronoberg',
       'Västmanland', 'Gävleborg', 'Västerbotten', 'Halland', 'Uppsala',
       'Dalarna', 'Norrbotten', 'Värmland', 'Blekinge', 'Södermanland',
       'Jämtland'], dtype=object)

In [6]:
import json

with open("assets/swedish_regions.geojson", "r") as file:
    json_data = json.load(file)


json_data.get("features")[0].get("properties")
    

{'admin_level': '4',
 'boundary': 'administrative',
 'name': 'Gotlands lÃ¤n',
 'ref:se:lÃ¤n:bokstav': 'I',
 'ref:se:lÃ¤nskod': '09',
 'type': 'multipolygon'}

In [7]:
properties = [feature.get("properties") for feature in json_data.get("features")]
region_codes = {
    property.get("name"): property.get("ref:se:länskod") for property in properties
}

region_codes

{'Gotlands lÃ¤n': None,
 'Kalmar lÃ¤n': None,
 'Blekinge lÃ¤n': None,
 'JÃ¤mtlands lÃ¤n': None,
 'VÃ¤sterbottens lÃ¤n': None,
 'Kronobergs lÃ¤n': None,
 'VÃ¤stra GÃ¶talands lÃ¤n': None,
 'VÃ¤rmlands lÃ¤n': None,
 'Dalarnas lÃ¤n': None,
 'SÃ¶dermanlands lÃ¤n': None,
 'JÃ¶nkÃ¶pings lÃ¤n': None,
 'Ã–stergÃ¶tlands lÃ¤n': None,
 'Stockholms lÃ¤n': None,
 'VÃ¤stmanlands lÃ¤n': None,
 'Uppsala lÃ¤n': None,
 'GÃ¤vleborgs lÃ¤n': None,
 'VÃ¤sternorrlands lÃ¤n': None,
 'Ã–rebro lÃ¤n': None,
 'Norrbottens lÃ¤n': None,
 'Hallands lÃ¤n': None,
 'SkÃ¥ne lÃ¤n': None}

In [8]:
from difflib import get_close_matches

get_close_matches("appel", ["ape", "apple", "peach", "puppy"], n = 1[0])
['apple', 'ape']

<>:3: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<>:3: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
C:\Users\milou\AppData\Local\Temp\ipykernel_23956\1269017890.py:3: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
  get_close_matches("appel", ["ape", "apple", "peach", "puppy"], n = 1[0])


TypeError: 'int' object is not subscriptable